In [9]:
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv('reviews.csv')

df = df.dropna()

# Sample data
texts = df['review_text']
labels = df['is_negative']

# Tokenize text
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

# Pad sequences
data = pad_sequences(sequences, maxlen=100)

# Convert labels to numpy array
labels = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

model.save('sentiment_model.h5')

Epoch 1/5


/Users/darlyn/.local/share/virtualenvs/sentiment_sense-cn05wPR5/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 17s 83ms/step - accuracy: 0.6086 - loss: 0.6510 - val_accuracy: 0.7431 - val_loss: 0.5303
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - accuracy: 0.8273 - loss: 0.4140 - val_accuracy: 0.7875 - val_loss: 0.4826
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 0.9033 - loss: 0.2690 - val_accuracy: 0.7644 - val_loss: 0.5111
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 99ms/step - accuracy: 0.9450 - loss: 0.1644 - val_accuracy: 0.7825 - val_loss: 0.5983
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 19s 97ms/step - accuracy: 0.9557 - loss: 0.1263 - val_accuracy: 0.7594 - val_loss: 0.7315
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.7705 - loss: 0.7486


Test Accuracy: 0.7593749761581421
